#Setting up


Importing libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
plt.style.use('fivethirtyeight')

Store the data

In [ ]:
#dito ilalagay yung csv file location
chart_path = '/content/DITO Historical Data.csv'

In [ ]:
trading_chart = pd.read_csv(chart_path, index_col = 'Date', parse_dates = True)

#optional if price is in thousands
if trading_chart['Price'].dtypes == 'O':
  trading_chart['Price'] = trading_chart['Price'].str.replace(',', '').astype(float)

trading_chart

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2021-02-05,14.54,14.10,14.60,13.84,58.10M,4.01%
2021-02-04,13.98,13.18,14.10,13.04,51.04M,3.56%
2021-02-03,13.50,14.20,14.44,13.50,85.59M,-1.17%
2021-02-02,13.66,12.72,13.66,12.72,69.17M,9.28%
2021-02-01,12.50,12.00,12.60,11.90,39.95M,4.17%
...,...,...,...,...,...,...
2012-05-08,3.00,2.98,3.10,2.88,566.00K,0.00%
2012-05-07,3.00,3.04,3.06,2.98,914.00K,-2.28%
2012-05-04,3.07,3.07,3.07,3.05,316.00K,-0.32%


#Simple Moving Average

In [ ]:
simple_moving_average = pd.read_csv(chart_path, index_col = 'Date', parse_dates = True)

#define a function for number of days you want to make for moving average

def set_days_SMA(n):

  #lalagyan ng moving average mo for this function only
  moving_average = []

  global trading_chart

  #calculation for the moving average
  for i in range(len(trading_chart)):
    if i > len(trading_chart) - n:
      moving_average.append(np.nan)
    else:
      moving_average.append(trading_chart['Price'][i:i+n].mean())

  #ilalagay na yung bagong dataset sa trading_chart
  new_set = 'Moving_Average_' + str(n)
  simple_moving_average[new_set] = moving_average
  return simple_moving_average

In [ ]:
set_days_SMA(5)
set_days_SMA(20)

,Price,Open,High,Low,Vol.,Change %,Moving_Average_5,Moving_Average_20
Date,,,,,,,,
2021-02-05,14.54,14.10,14.60,13.84,58.10M,4.01%,13.636,12.328
2021-02-04,13.98,13.18,14.10,13.04,51.04M,3.56%,13.128,12.257
2021-02-03,13.50,14.20,14.44,13.50,85.59M,-1.17%,12.716,12.193
2021-02-02,13.66,12.72,13.66,12.72,69.17M,9.28%,12.368,12.131
2021-02-01,12.50,12.00,12.60,11.90,39.95M,4.17%,11.840,12.118
...,...,...,...,...,...,...,...,...
2012-05-08,3.00,2.98,3.10,2.88,566.00K,0.00%,3.062,NaN
2012-05-07,3.00,3.04,3.06,2.98,914.00K,-2.28%,NaN,NaN
2012-05-04,3.07,3.07,3.07,3.05,316.00K,-0.32%,NaN,NaN


#Exponential Moving Average

In [ ]:
exponential_moving_average = pd.read_csv(chart_path, index_col = 'Date', parse_dates = True)

#define a function for number of days you want to make for moving average

def set_days_EMA(n):

  global trading_chart
  #lalagyan ng moving average mo for this function only
  moving_average = []
  moving_average.append(trading_chart['Price'][len(trading_chart) - 1])

  #calculation for the moving average
  for i in range(len(trading_chart)-1):
    expo_ave = (trading_chart['Price'][len(trading_chart) - 2 - i] * (2/(n+1))) + (moving_average[0] * (1 - (2/(n+1))))
    moving_average.insert(0, expo_ave)

  #ilalagay na yung bagong dataset sa trading_chart
  new_set = 'Moving_Average_' + str(n)
  exponential_moving_average[new_set] = moving_average

  return exponential_moving_average

In [ ]:
set_days_EMA(20)
set_days_EMA(5)

,Price,Open,High,Low,Vol.,Change %,Moving_Average_20,Moving_Average_5
Date,,,,,,,,
2021-02-05,14.54,14.10,14.60,13.84,58.10M,4.01%,12.326180,13.685916
2021-02-04,13.98,13.18,14.10,13.04,51.04M,3.56%,12.093146,13.258874
2021-02-03,13.50,14.20,14.44,13.50,85.59M,-1.17%,11.894530,12.898311
2021-02-02,13.66,12.72,13.66,12.72,69.17M,9.28%,11.725533,12.597467
2021-02-01,12.50,12.00,12.60,11.90,39.95M,4.17%,11.521905,12.066200
...,...,...,...,...,...,...,...,...
2012-05-08,3.00,2.98,3.10,2.88,566.00K,0.00%,3.118316,3.049877
2012-05-07,3.00,3.04,3.06,2.98,914.00K,-2.28%,3.130770,3.074815
2012-05-04,3.07,3.07,3.07,3.05,316.00K,-0.32%,3.144535,3.112222


#Support and Resistance

In [ ]:
def create_resistance(MA):

  '''diff = []
  for i in range(50):
    if trading_chart['Open'][i] <= trading_chart['Price'][i]:
      diff.append((trading_chart['Open'][i] - simple_moving_average['Moving_Average_' + str(MA)][i]))
    else:
      diff.append((trading_chart['Price'][i] - simple_moving_average['Moving_Average_' + str(MA)][i]))

  resistance = []
  average_range = sum([(abs(trading_chart['Price'][x] - trading_chart['Open'][x])) for x in range(50)]) / 50

  for i in range(len(trading_chart)):
      resistance.append(simple_moving_average['Moving_Average_' + str(MA)][i] + ((abs(sum(diff)) * 2)/len(diff)) + average_range)


  simple_moving_average['MA_' + str(MA) + '_Resistance'] = resistance
  return simple_moving_average'''

  simple_moving_average['MA_' + str(MA) + '_Resistance'] = simple_moving_average['Moving_Average_' + str(MA)]
  simple_moving_average['MA_' + str(MA) + '_Support'] = simple_moving_average['Moving_Average_' + str(MA)]
  simple_moving_average['STD_' + str(MA)] = trading_chart['Price'][::-1].rolling(MA).std()

  for i in range(len(simple_moving_average)-20):
    simple_moving_average['MA_' + str(MA) + '_Resistance'][i] = (2*simple_moving_average['STD_' + str(MA)][i]) + simple_moving_average['Moving_Average_' + str(MA)][i]
    simple_moving_average['MA_' + str(MA) + '_Support'][i] = simple_moving_average['Moving_Average_' + str(MA)][i] - (2*simple_moving_average['STD_' + str(MA)][i])

  return simple_moving_average

In [ ]:
create_resistance(20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Price,Open,High,Low,Vol.,Change %,Moving_Average_5,Moving_Average_20,MA_20_Resistance,MA_20_Support,STD_20,MA_5_Resistance,MA_5_Support,STD_5
Date,,,,,,,,,,,,,,
2021-02-05,14.54,14.10,14.60,13.84,58.10M,4.01%,13.636,12.328,14.202568,10.453432,0.937284,15.134372,12.137628,0.749186
2021-02-04,13.98,13.18,14.10,13.04,51.04M,3.56%,13.128,12.257,13.867820,10.646180,0.805410,14.805522,11.450478,0.838761
2021-02-03,13.50,14.20,14.44,13.50,85.59M,-1.17%,12.716,12.193,13.605025,10.780975,0.706013,14.358778,11.073222,0.821389
2021-02-02,13.66,12.72,13.66,12.72,69.17M,9.28%,12.368,12.131,13.403377,10.858623,0.636188,13.914764,10.821236,0.773382
2021-02-01,12.50,12.00,12.60,11.90,39.95M,4.17%,11.840,12.118,13.328404,10.907596,0.605202,12.910701,10.769299,0.535350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-05-08,3.00,2.98,3.10,2.88,566.00K,0.00%,3.062,NaN,NaN,NaN,NaN,3.062000,3.062000,0.066483
2012-05-07,3.00,3.04,3.06,2.98,914.00K,-2.28%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-04,3.07,3.07,3.07,3.05,316.00K,-0.32%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Visualizing the Data

In [ ]:
def plot_figure_SMA():

  figure = go.Figure(
      data = [
              go.Candlestick(
                  x = trading_chart.index,
                  low = trading_chart['Low'][:260],
                  high = trading_chart['High'][:260],
                  close = trading_chart['Price'][:260],
                  open = trading_chart['Open'][:260],
                  increasing_line_color = 'green',
                  decreasing_line_color = 'red'
              ),
             go.Scatter(
                  x = trading_chart.index,
                  y = trading_chart['Price'][:260],
                  mode = 'lines',
                  line_color = 'black',
                  name = 'Moving Average'
              ),
              go.Scatter(
                  x = trading_chart.index,
                  y = simple_moving_average['Moving_Average_20'][:260],
                  mode = 'lines',
                  line_color = 'grey',
                  name = 'Support',
                  opacity = 0.3
              ),
              go.Scatter(
                  x = trading_chart.index,
                  y = simple_moving_average['MA_20_Resistance'][:260],
                  mode = 'lines',
                  line_color = 'purple',
                  name = 'Resistance',
                  opacity = 0.3
              ),
              go.Scatter(
                  x = trading_chart.index,
                  y = simple_moving_average['MA_20_Support'][:260],
                  mode = 'lines',
                  line_color = 'purple',
                  name = 'Resistance',
                  opacity = 0.3
              ),
              go.Scatter(
                  x = trading_chart.index,
                  y = exponential_moving_average['Moving_Average_5'][:260],
                  mode = 'lines',
                  line_color = 'orange',
                  name = 'Moving Average_5'
              )

      ],
      layout = go.Layout(
                  autosize=False,
                  width=1500,
                  height=900
      )

  )

  figure.show()

In [ ]:
plot_figure_SMA()